# Novelty analysis
In this notebook, we'll look at the trade-offs between novelty and accuracy in semantic matching.

In [1]:
import logging
logging.getLogger().setLevel(logging.INFO)

import takco
import pandas as pd

conf = takco.config.parse('resources/config-dbpedia.toml')
tables = takco.TableSet.load('output/t2d-v2-baseline-2/1-link/*')

t2dv2 = takco.config.build('t2d-v2', conf)
scored_tables = tables.score(t2dv2, keycol_only=True)
scored_tables.tables.persist()

takco.preview( t for t in scored_tables if any(t.get('gold', {}).values()) )

INFO:root:Loading data from resources/t2d_fix.csv
INFO:root:Read 512 tables from data/t2d-v2/tables
INFO:root:Read 512 entity tables from data/t2d-v2/instance
INFO:root:Loaded 514 annotated tables


?,0,1,2
∈,Book,,
0,,author,
,Title,Author,Source
,Adventures of Huckleberry Finn ❌ 💡,Mark Twain,ALA [11]
,The Adventures of Super Diaper Baby ❌ 💡,Dav Pilkey,ALA [47]
,The Adventures of Tom Sawyer ❌ 💡,Mark Twain,ALA
,Alice series ❌ 💡,Phyllis Reynolds Naylor,ALA [2]
,All the King's Men ❌ 💡,Robert Penn Warren,Rad
?,0,1,2
∈,,Newspaper,


In [21]:
db = takco.config.build('dbpedia_t2ksubset', conf)

novelty_tables = scored_tables.triples().novelty(db)
novelty_tables.tables.persist()

len(list(novelty_tables))

235

In [22]:
report = novelty_tables.report(keycol_only=True)

display(pd.DataFrame.from_dict(report['scores'], orient='index').style.set_caption('Predictions:'))

def reform_dict(dictionary, t=tuple(), reform={}):
    for key, val in dictionary.items():
        t = t + (key,)
        if isinstance(val, dict) and all(isinstance(v, dict) for v in val.values()):
            reform_dict(val, t, reform)
        else:
            reform.update({t: val})
        t = t[:-1]
    return reform

display()
pd.DataFrame.from_dict(reform_dict(report['novelty']), orient='index').style.set_caption('Extractions:')

INFO:root:Collected 26104 gold and 24061 pred for task entities
INFO:root:Collected 434 gold and 157 pred for task properties
INFO:root:Collected 235 gold and 235 pred for task classes


,precision,recall,f1-score,support,predictions
entities,0.867670,0.799762,0.832333,26104,24061
properties,0.777070,0.281106,0.412860,434,157
classes,0.740426,0.740426,0.740426,235,235
